# NoteBook To Check Point or Polygon Inside Polygon

The objective of this notebook is to search a set of points or polygons for spatial overlap with a largerset

By Cascsde Tuholske 2019-01-17

#### Notes
1. Points should be dialated to circles to prevent one falling just outside the larger polygon set (see Lusaka)

### Depedencies

In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry import mapping
from shapely.geometry import Polygon
from shapely.geometry import shape
import ast
from shapely.geometry import mapping
import rasterio
import copy

### Functions

In [2]:
def load_points (file):
    """ This function loads a csv 
    of points and turns it into shapely points"""
    df = pd.read_csv(file)

    # creating a geometry column 
    geometry = [Point(xy) for xy in zip(df['lon'], df['lat'])]

    # Coordinate reference system : WGS84
    crs = {'init': 'epsg:4326'}

    # Creating a Geographic data frame 
    point_gdf = gpd.GeoDataFrame(df, crs=crs, geometry=geometry)
    
    return point_gdf

In [3]:
def point_buffer(gpd_df, raduis):
    "Function to make a shapely polygon buffer around a point"
   
    # AGU 2018-12-04 - radius set to ~250m at the equator 
    #radius = 250*1/(111*1000)

    # new_gpd_df = gpd.GeoDataFrame()
    gpd_df.rename(columns={'geometry':'old_geom'}, inplace=True)
    
    arr = []
    
    for point in gpd_df['old_geom']:
        buffer = point.buffer(radius)
        arr.append((buffer))
    
    gpd_df['geometry'] = arr
    
    return gpd_df

In [4]:
def poly_point (poly, point):
    """
    This function will check if points are inside polygons if given two gpd dataframes with points and polygons
    Returns point ids, point geometry, polygon index # and polygon geometry
    """
    
    out_arr = [] #return an array <<< ---------------- ASK RYAN IF BETTER DO USE DICT 
    
    for index_point, row_point in point.iterrows():
        for index_poly, row_poly in poly.iterrows():
            if row_point['geometry'].within(row_poly['geometry']):
                point_id = row_point['id']
                point_geom = mapping(row_point['geometry']) # makes a dict w/ keys : type and cood
                poly_id = index_poly
                poly_geom = mapping(row_poly['geometry']) # makes a dict w/ keys : type and cood
                
                out_arr.append((point_id, 
                                point_geom, 
                                poly_id, 
                                poly_geom))

    return out_arr

# Note 2018-11-30 update arr to gpd_df 

In [5]:
def poly_overlap (point_buffer, poly_raster):
    """
    This function will check if point buffers intersect with polygons 
    if given two gpd dataframes with point buffers and polygons
    Returns point ids, point geometry, polygon index #, and polygon geometry in a geopandas DF.
    It goes faster if smaller list goes first
    
    Change TOWN & CITY ARRA
    """
    
    # make arrays to fill
    country_arr = []
    osm_name_arr = []
    osm_type_arr = []
    lat_arr = []
    lon_arr = []
    osm_id_arr = [] 
    FID_arr = [] 
    poly_geom_arr = []
    #point_geom_arr = []
    
    for index_point_buffer, row_point_buffer in point_buffer.iterrows():
        for index_poly_raster, row_poly_raster in poly_raster.iterrows():
            if row_point_buffer['geometry'].intersects(row_poly_raster['geometry']):
                country = row_point_buffer['country']
                osm_name = row_point_buffer['osm_name'] 
                osm_type = row_point_buffer['osm_type']
                lat = row_point_buffer['lat']
                lon = row_point_buffer['lon']
                osm_id = row_point_buffer['osm_id']
                poly_id = row_poly_raster['FID']
                poly_geom = shape(mapping(row_poly_raster['geometry'])) # make polygon
                #point_geom = row_point_buffer['old_geom']

                country_arr.append((country))
                osm_name_arr.append((osm_name))
                osm_type_arr.append((osm_type))
                lat_arr.append((lat))
                lon_arr.append((lon))
                osm_id_arr.append((osm_id))
                FID_arr.append((poly_id))
                poly_geom_arr.append((poly_geom))
                #point_geom_arr.append((point_geom))
    
    # put results into a geopandas df
    new_gpd_df = gpd.GeoDataFrame()
    new_gpd_df['osm_id'] = osm_id_arr
    new_gpd_df['FID'] = FID_arr
    new_gpd_df['geometry'] = poly_geom_arr
    new_gpd_df['country'] = country_arr           
    new_gpd_df['osm_name'] = osm_name_arr
    new_gpd_df['osm_type'] = osm_type_arr
    new_gpd_df['lat'] = lat_arr
    new_gpd_df['lon'] = lon_arr
    #new_gpd_df['point_geom'] = point_geom_arr
    
    
    return new_gpd_df

# Note 2018-11-30 update arr to gpd_df 
# Note 2019-04-30 updated to use osm_name instead of city/town for the OSM name

### Load Data

In [6]:
import os
os.getcwd()

'/tana-crunch/cascade/projects/Pop-ERL/notebooks/jupyter/ERL19'

In [7]:
# File Paths
temp_data = '../../../temp_data/ERL19/'
erl_v2_data  = '../../../temp_data/ERL19v2/'

In [8]:
# Load Polygons

# Load Points
points = gpd.read_file(erl_v2_data+'20190430_osm_All.shp')
print(len(points))

# Load Polygons
polygons = gpd.read_file(temp_data+'WPE_1KM_2016_Pop_Clip_1500c300_A.shp')
print(len(polygons))

#Out File Name
out = 'WPE_1KM_2016_Pop_Clip_1500c300_A_polypoints'

9813
15000


### Buffer Points

In [9]:
# AGU 2018-12-04 - radius set to ~250m at the equator 

radius = 250*1/(111*1000)
radius

0.0022522522522522522

In [10]:
osm_buffer_gdf = point_buffer(points, radius)

In [11]:
#osm_buffer_gdf.head(6)

In [12]:
# drop point geom and keep buffered

osm_buffer_gdf = osm_buffer_gdf.drop(['old_geom'], axis=1)
osm_buffer_gdf.head(6)

,country,osm_id,osm_name,lat,lon,osm_type,geometry
0,Algeria,89369215,Tamanrasset,22.785454,5.532446,town,"POLYGON ((5.534698752252252 22.7854543, 5.5346..."
1,Algeria,89980948,In Salah,27.195033,2.482613,town,"POLYGON ((2.484865452252253 27.1950331, 2.4848..."
2,Algeria,252600742,Boumerdès,36.758882,3.470596,town,"POLYGON ((3.472848252252252 36.758882, 3.47283..."
3,Algeria,253167052,Thenia,36.724986,3.556935,town,"POLYGON ((3.559187252252252 36.724986, 3.55917..."
4,Algeria,253167208,Zemmouri,36.786406,3.601221,town,"POLYGON ((3.603473152252253 36.7864064, 3.6034..."
5,Algeria,253291208,Lakhdaria,36.563944,3.596907,town,"POLYGON ((3.599159252252252 36.5639442, 3.5991..."


In [13]:
polygons.head()

,FID,geometry
0,0,"POLYGON ((19.228423 -34.42911, 19.228423 -34.4..."
1,1,"POLYGON ((9.858949000000001 37.337643, 9.86793..."
2,2,"POLYGON ((18.878078 -34.159614, 18.878078 -34...."
3,3,"POLYGON ((19.255372 -34.40216, 19.300288 -34.4..."
4,4,"POLYGON ((18.392985 -34.150631, 18.410952 -34...."


#### Change CRS if needed

In [ ]:
polygons.crs

In [ ]:
osm_buffer_gdf.crs

In [ ]:
# For GHS CRS

crs = {'ellps': 'WGS84',
 'lon_0': 0,
 'no_defs': True,
 'proj': 'moll',
 'units': 'm',
 'x_0': 0,
 'y_0': 0}

In [ ]:
raster_polygons.crs = crs

In [ ]:
raster_polygons_crs = raster_polygons.to_crs(osm_buffer_gdf.crs)

In [ ]:
# For GHS make FID and reset crs
FID = list(range(len(raster_polygons)))
raster_polygons['DN'] = FID
raster_polygons.columns.values[0] = "FID"

In [ ]:
raster_polygons.head()

### Intersection

In [ ]:
# run poly_overlap

import time
checkpoint = time.time()

poly_overlap_out = poly_overlap(osm_buffer_gdf, polygons)

print("elapsed time is: {}s".format(time.time()-checkpoint))
print(len(poly_overlap_out))

In [15]:
print(len(poly_overlap_out))

3627


In [16]:
poly_overlap_out.tail(20)

,osm_id,FID,geometry,country,osm_name,osm_type,lat,lon
3607,2147483647,14089,"POLYGON ((-1.765307 6.13002, -1.756323 6.13002...",Ghana,Obuasi,city,6.211683,-1.689611
3608,2147483647,12233,"POLYGON ((-1.819206 4.944238, -1.810223 4.9442...",Ghana,Sekondi-Takoradi,city,4.927456,-1.749022
3609,27564980,14370,"POLYGON ((-10.541889 6.219852, -10.532906 6.21...",Liberia,Monrovia,city,6.328034,-10.797788
3610,27565005,12646,"POLYGON ((5.834477 5.330515, 5.84346 5.330515,...",Nigeria,Warri,city,5.517000,5.750000
3611,27565065,12018,"POLYGON ((7.16399 4.692709, 7.16399 4.674742, ...",Nigeria,Port Harcourt,city,4.767658,7.018853
3612,27565107,14048,"POLYGON ((7.451453 6.058154, 7.451453 6.049171...",Nigeria,Enugu,city,6.449983,7.500001
3613,27565124,14871,"POLYGON ((3.337149 6.417482, 3.337149 6.408499...",Nigeria,Lagos,city,6.455057,3.394179
3614,251608378,12237,"POLYGON ((8.340788999999999 4.935255, 8.340788...",Nigeria,Calabar,city,4.979600,8.337360
3615,501193829,12315,"POLYGON ((7.909596 4.989154, 7.909596 4.971188...",Nigeria,Uyo,city,5.032245,7.924791
3616,501482915,13471,"POLYGON ((5.735662 5.860524, 5.735662 5.851541...",Nigeria,Sapele,city,5.894053,5.676660


In [17]:
poly_overlap_out.to_file(erl_v2_data+out+'.shp', driver='ESRI Shapefile')

/home/cascade/miniconda3/envs/geo/lib/python3.6/site-packages/geopandas/io/file.py:108: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():
